# Communicating between SuperCollider and Python

One may ask the question why there are so many different programming languages in the first place is valid.
One of the reasons is that certain programming languages follow ceration aproaches that fit certain domain specific needs.

The programming language *sclang* of SuperCollider platform is an *interpreted* language which means [each instruction gets dynamically evaluated](https://stackoverflow.com/a/3265602/3475778).
Of course there is the question how *sclang* itself has been written which is C++, a *compiled* language.
Basically compiled languages have the advantage of being fast where interpreted languages allow for a more dynamic aproach which comes handy during live situations but are therefore slower.

As audio processing is time critical (see [real time computing](https://en.wikipedia.org/wiki/Real-time_computing)) we need a fast language (C++) to generate the audio signal but we want also a language on a more abstract level to control the audio generation and not care about e.g. memory allocation, which *sclang* delivers.
A different aproach is used by e.g. [openFrameworks](https://openframeworks.cc/) in which one programs directly in C++ but *openFrameworks* delivers convenience function for often used tasks - this aproach is often called a library or framework and is not a new programming language like *sclang*.

As *sclang* itself is not fast enough to generate audio signals we need to send send control commands to the audio generating server *scsynth* which is written in C++.
In order to communicate between both entities *sclang* and *scsynth* uses *OSC* (Open Sounc Control) which defines a protocol to send data between both entities.

![Drawing](scsynth_sclang.svg)

As we now have lurked into the topic of the motivation behind the multitude of programming languages we now introduce a new programming language [Python](https://www.python.org/).
The first paragraph of the [Wikipedia article about Python](https://en.wikipedia.org/wiki/Python_(programming_language)) says

> Python is an interpreted high-level general-purpose programming language. Python's design philosophy emphasizes code readability with its notable use of significant indentation. Its language constructs as well as its object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.

Sounds great, but as it is an interpreted language it is still not fast enough for audio generation like *scsynth* and as it is not designed specifically to control audio synthesis like *sclang* but nonetheless it has the advantage that a lot of people adapt a variety of applications and libraries for Python.
In particular Python gained popularity within the last years as it is the de-facto standard for data analysis as lots Data Science and Machine Learning applications rely on the simplicity and readability of Python.
We could implement these algorithms in sclang as both languages are turing complete but this is a big task as these algorithms can be tricky to implement properly so we need to find a way to communicate between both languages what this chapter is about.

In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube-nocookie.com/embed/RPQD7-AOjMI" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

## OSC

A message is send from a sender to a receiver - so when we want to implement a messanging system between SuperCollider and Python using OSC we need to consider who is the sender (the client) and who is the receiver (the server).

But before we can start building the messanging infrastructure we need to teach Python the OSC protocol.
By doing some research we find the Python library [python-osc](https://github.com/attwad/python-osc) which implements the OSC protocol for Python so we do not need to take care of this.
Now it comes down to reading documentation.

### From Python to SuperCollider

We start by sending an OSC message from Python to SuperCollider which controls the frequency of a synth.
It is good practice to set up the receiver first before sending a message from the client so we focus on the SuperCollider part first.

![Python as a OSC client](python_client.svg)

#### SuperCollider

We start by booting the scsynth server

```supercollider
s.boot;
```

and adding a SynthDef to it

```supercollider
SynthDef(\foo, {|out|
    var sig = Saw.ar(freq: \freq.kr(200));
    var env = EnvGen.kr(Env.linen(
        sustainTime: 0.2,
        releaseTime: 0.2,
    ), doneAction: Done.freeSelf);
    Out.ar(out, sig*env*\amp.kr(0.2)!2);
}).add;
```

and testing it

```supercollider
Synth(\foo, [\freq, 400]);
```

Now it is time to take a look at the [documentation of the OSC communication in SuperCollider](https://doc.sccode.org/Guides/OSC_communication.html).

A good starting point is to verify the port of the *sclang* instance by executing

```supercollider
NetAddr.langPort;
-> 57120
```

This port can change if the port is already taken by e.g. another sclang instance so it is good practice to verify this port as it will be important when writing the Python section.

We can use [`OSCdef`](https://doc.sccode.org/Classes/OSCdef.html) to register a function which will be called when we receive an OSC message on a *path* which is like a sub-URL.
A parameter of the function we need to provide is the actually received message - we call this kind of function a [*callback function*](https://developer.mozilla.org/en-US/docs/Glossary/Callback_function) which are often used with an [event loop](https://developer.mozilla.org/en-US/docs/Web/JavaScript/EventLoop) like in JavaScript.

```supercollider
OSCdef(key: \myOscDef, func: {|msg|
    # msg[0] is the path that the message was received on
    Synth(\foo, [\freq, msg[1]]);
}, path: "/my_path");
```

We have now setup a SuperCollider server which listens on OSC messages on channel `/my_path` and expects a frequency value in this message which will be used to play our synth.

#### Python

Taking a look at the documentation of *python-osc* we find an [example](https://github.com/attwad/python-osc#simple-client) which implements an OSC client so we only need to adjust the code for our needs.

As always we need to import the libraries we want to use before we can use them.
If this part fails check that you have installed all necesarry dependencies - see {ref}`install-dependencies`.

In [ ]:
import time

import numpy as np
from pythonosc import udp_client

np.random.seed(42)

Now we need to specify the address of the OSC server which we want to send our message to.
The IP address `127.0.0.1` is our own machine (also often called `localhost`) and we identified the port via `NetAddr.langPort;` before.

In [ ]:
osc_client = udp_client.SimpleUDPClient(address='127.0.0.1', port=57120)

Now we simply send a message from Python to OSC on channel `/my_path`.
If everything worked correctly we should hear a sound from SuperCollider.

In [ ]:
for i in range(10):
    osc_client.send_message("/my_path", np.random.randint(100, 1000))
    time.sleep(0.2)

#### Debugging the communication

If no sound was played there are some steps that can help you to find the issue.

* Check ports and adresses twice! Prefer copy & pasting over typing as transposed digits *("Zahlendreher")* are notorious for wasting precios lifetime
* Do not miss the slash in the osc path: For SuperCollider and for Python
* Use [`OSCFunc.trace(false, true);`](https://doc.sccode.org/Classes/OSCFunc.html#*trace) in SuperCollider to dump all incoming OSC messages. If you do not receive any kind of messages here there is something wrong with the addresses.